# Foundations of Distributed Deep Learning using Azure Machine Learning

## Introduction

In recent years, we have witnessed a significant surge in the size and complexity of Large Language Models (LLMs) and their associated technologies. These advancements have revolutionized various fields, from natural language processing to artificial intelligence, enabling machines to understand and generate human-like text with remarkable accuracy. However, this progress comes with its own set of challenges. One of the most pressing issues is the immense computational power required to train these models. It has become evident that a single GPU, regardless of its capabilities, is insufficient to train these models within a reasonable timeframe that aligns with market demands.

To address this challenge, organizations are increasingly adopting the use of multiple GPUs in a single training run, a technique known as Distributed Deep Learning. This approach leverages the combined power of several GPUs to accelerate the training process, making it feasible to handle the vast amounts of data and complex computations involved in training LLMs. By distributing the workload across multiple GPUs, organizations can achieve faster training times and improve their time-to-market, which is crucial in the competitive landscape of AI development.

In this lab, we will walk through the concepts that allow a Deep Learning (DL) framework to utilize multiple GPUs during a single training run. We will begin with simple jobs to introduce the fundamental concepts of distributed computing. These initial exercises will help you understand how tasks are divided and managed across multiple GPUs, ensuring efficient utilization of resources.  We will then see how everything fits together for a training run of the MNIST dataset and model.

It is important to note that this lab will focus on the mechanics of using multiple GPUs for distributed training rather than delving into specific deep learning techniques. The goal is to equip you with the knowledge and skills needed to set up and execute distributed training runs, laying a solid foundation for further exploration and application of advanced deep learning models.

## Prerequisties
In order to run this lab you will need:

* An Azure Machine Learning (AML) workspace 
* The Azure Commnad-Line Interface (CLI), available for download [here](https://learn.microsoft.com/en-us/cli/azure/)
* A basic understanding of Python and [PyTorch](https://pytorch.org/).  (Note: You do not need PyTorch installed on your local machine as all execution will take place on AML machines.)

## Tools Used
* Azure Machine Learning jobs and [serverless compute](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-serverless-compute?view=azureml-api-2&tabs=python)
* PyTorch along with the [PyTorch Lightning](https://lightning.ai/docs/pytorch/stable/starter/introduction.html) library
* The [Azure CLI's AML support](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-train-model?view=azureml-api-2&tabs=python)

We will do most of our work from a command-line shell so go ahead and open one now.




## Setup 

In order to setup for this lab you will need to:

1.  Open a command-line shell
2.  Install the Azure CLI (see above)
3.  Modify the following values and copy them into the shell.  These will allow the AML CLI to run the jobs in your workspace.
4.  Make sure that the shell is open in the top level directory of this lab's repository 

The values in step #3 above can be found on the top-right corer, next to your profile initials.

In [ ]:
export SUBSCRIPTION_ID="<YOUR SUBSCRIPTION ID"
export RESOURCE_GROUP="<RESOURCE GROUP IN WHICH YOUR WORKSPACE IS LOCATED"
export WORKSPACE_NAME="<AML WORKSPACE NAME"



## Part 1 - How to Use Multiple Compute Units is a Single Job

In this section we will get accquainted with the environment variables that AML assigns to different compute units in a single job.  The purpose of this assignment is so that different compute units (CPUs, GPUs, etc.) know where and how to talk to other units.  By making sure that each unit is uniquely identifiable, the DL framework can then assign different parts of the training to it.  We will begin by using CPU-based machines as they are cheap and plentiful and can easily explain the required points.

In order to run an AML job from the command line, we need a _job description_ file with a `.yaml` extension.  Open the `jobs/simple_environment.yaml` file from the lab's repository in a text editor of your choice.  Since this is a fairly simple job that only uses OS commands and no Python code, we are able to provide the commands that AML will run directly in the job file.

There are a few interesting point to note about this file:

#### Job Command

In line 2, prefixed by the `command:` key, we have the actual command to be run on AML's compute resources.  In this job we ask for environment variables of the running process, sort them and then filter them out to only show the ones we're interested in.  

#### Environment Image

When AML runs jobs, it does so in the context of _environments_, which are Docker containers.  By encapsulating the specific libraries and tools required for each job, AML can re-use the same compute resources across multiple jobs.  You can learn more about AML environments [here](https://learn.microsoft.com/en-us/azure/machine-learning/concept-environments?view=azureml-api-2).  For this lab we will be using pre-prepared environments.  

In line 4, we specify our environment as the latest Python Docker container without any modifications.  AML will run this container and run our command (line 2) inside it.  

#### Distribution 

In line 5 we have the _distribution_ section, which instructs AML on how to run the distributed training job.  Here we see that we ask for a `pytorch` distribution type and 3 processes on each node.  The distribution type determines the type of environment variables that AML makes available to the processes makin up the training job (see below).  Current possible values are `pytorch' and 'mpi' and they differ in the number and names of these environment variables.  For this job, we instruct AML to use 3 training processes on each node, for a total of 3 x (number of nodes) processes.

#### Resources

Immediately below the distribution, on line 8, we have our resource declaration.  We ask for 2 nodes of type DS3_V2, which is a VM with 4 CPU cores.  Having asked for 3 processes per node in the distribution section above, it follows that we will have a total of 2 x 3, or 6, processes across all nodes, each running the command.  The use of the resources section also implies that we using _serverless_ compute - that is, the resources are allocated _on-demand_ and are shut down after the job completes.  


### Running the Code
From your shell, run the following command:







In [ ]:
az ml job create --file jobs/simple_environment.yaml --web \
    --resource-group ${RESOURCE_GROUP} --workspace-name ${WORKSPACE_NAME}

AML will now start running the job and open a web browser to a status screen reporting the status of the job.  In the background the new VMs are being allocated and the job run.  In a few minutes your job should report its status and `Running` and the `Completed`.

Once the job has completed, click the `Outputs and logs` below the job name and open the `user_logs` folder.  Your screen should resemble the following:



![](./pics/outputs_and_logs.png)




As we can see, we have 6 files, labeled `std_log_process_0.txt` through `std_log_process_5.txt`, corresponding to the 6 processes we were expecting.  Each such file is the standard output of the relevant process and AML has collected it from all of the nodes in the job automatically.  Looking inside these files, we can find some interesting environment variables:

* RANK:  This is the index of the current process across *all* processes, regardless on what node they run - in other words, a *global* rank
* LOCAL_RANK:  This is the index of the current process across all of the processes on its *current* noce
* NODE_RANK:  Similar to process ranks, each node has an index across all nodes of the job
* MASTER_ADDR and MASTER_PORT  - The IP address and port to use for communication (see below)
* WORLD_SIZE  - The total number of processes in the job

So for our 2-node, 3-process-per-node job we have the following process layout:

![](./pics/ranks.png)

**Exercise**:  Go through some of the log files and make sure you understand the global and local ranks and on what they nodes they ran.

AML further provides the AZUREML_NODE_COUNT environment variable so that we can easily compute the global and local rank for each process in the job.  By assigning each process a unique RANK it is now possible to run the *same* script multiple times across multiple compute units and nodes and have the processes work together to complete a task in parallel.  We will see an example in the next section.






## Part 2 - Communicating Between Ranks of a Single Job

In this section we will see how to communicate between two ranks of a single job.  We will run two nodes with a single rank each and pass a message from rank 0 to rank 1 using PyTorch's `distributed` package.  Once more, we will use cheap CPU machines to perform this task and move to GPUs in the next section.

We will now be using both a `.yaml` file and a python script.  Open `jobs/simple_node_communication.yaml` and `src/simple_node_communication.py` from the lab's repository in a text editor of your choice.  

The .yaml file should look familiar with a few changes:

*  We are running 4 processes in the job, with 4 nodes and a single process on each.  Again, we use serverless compute.
*  We change the environment to use a container that has been prepared by Microsoft with CUDA and PyTorch.  

The python file is much more interesting.  Let's go over the sections denoted by each numbered comment:

#### Comment 1
As we've seen before, AML will give each instance of a running command or script a set of environment variables.  In this section we extract the (global) rank as well as the world size, convert them to int and print them to the standard output, where we can then look at them in the AML job's status window.

#### Comment 2
In this section we initialize PyTorch's a distributed process group.  This construct, from PyTorch's `distributed` package sets up the necessary scaffolding for individual ranks to communicate.  As we are using CPUs for our compute units we choose the `Gloo` (see [here](https://github.com/facebookincubator/gloo)) backend for communication.  Gloo is a _collective communications library_, which is a set of routines for compute units to efficiently communicate between each other.  For example, it implements algorithms for one compute unit to broadcast to all others or for synchronizing the contents of all compute units using a specific mathematical operation - such as when averaging out the different gradients stored all compute units together.


#### Comment 3
In this section we prepare the object we want to send between ranks.  Specifically, we _broadcast_ a string - in other words, a list of characters - from rank 0 to all other ranks.  We first initialize a list with a None value that will be the initial value held by each rank.  We will want this value to get _overriden_ in ranks other than 0.  Next, we replace the None with a string value only in rank 0.


#### Comment 4
In this section we perform the actual call to the communications routine.  We pass the object we wish to send (a list, in this case, hence the _object_list_ suffix on the function name) and the source rank.  As each rank executes the same code, this function has the effect of _sending_ on rank 0 and _receiving_ on all other ranks, performing a _one-to-many send_ or _broadcast_.  We then print the first element of the list on each rank and verify that all ranks have in fact received the message.

#### Comment 5 
In this section we shut down the distributed communications group and clean up our resources.



### Running the Code
From your shell, run the following command:



In [ ]:
az ml job create --file jobs/simple_node_communication.yaml --web \
    --resource-group ${RESOURCE_GROUP} --workspace-name ${WORKSPACE_NAME}

**Exercise**:

1.  Change the number of nodes and processes per node and re-run the job.  Verify that the various rank layouts can receive broadcasts.

2.  Change the code to send the message from a different rank

By using broadcast and other collective communications primitives we are able to have individual ranks communicate as part of a single job.  So far, we have used the Gloo backend and CPUs to do this communication.  In the next section we will switch to GPUs.

## Part 3 - Communicating Between Ranks of a Single Job on GPUs

In this section we will once more run a job where two ranks communicate.  This time, though, we will use GPUs as our compute units and [NCCL](https://developer.nvidia.com/nccl), the Nvidia Collective Communications Library as the backend.  We will use _low priority_ (or _spot_) machines for our cluster in order to save on costs.

We will once more be using both a `.yaml` file and a python script.  Open `jobs/simple_gpu_communication.yaml` and `src/simple_gpu_communication.py` from the lab's repository in a text editor of your choice.  

The .yaml file now contains a new key:  `queue_settings` with a sub-key of `job_tier` and a value of `spot`.  AML will now try to allocate the GPU machines from the excess capacity available in the data center, thus saving on costs.  Note that this script may fail to allocate the nodes on the first try and/or may require multiple execution until these machines are available.

Let's look at the python file.  It should seem very similar to the last, CPU-based version with a few small changes.

#### Comment 1
We once more extract the rank and world size that is provided by AML.  This time we also ask for the _local rank_, as we'll be needing it for choosing a GPU.

#### Comment 2
We initialize the distributed process group again, this time specifying `NCCL` as the backend.  This implies that we will be running the communications code on the GPU and therefore each process should be running on a single GPU.  

#### Comment 3
We now prepare the message - a torch Tensor object - to send to the other ranks.  Since both the message and the communications code needs to run on the GPU we first set the current CUDA device - i.e., the GPU that will be used unless explicitly specified otherwise.  On multi-GPU machines this means that different ranks will need a different notion of what 'current' means.  So we need to use the **local** rank (i.e., the index of the rank on the current node) to decide what GPU will be the current one for the rank.  We then upload the data to this current GPU.

### Running the Code
From your shell, run the following command:


In [ ]:
az ml job create --file jobs/simple_gpu_communication.yaml --web \
    --resource-group ${RESOURCE_GROUP} --workspace-name ${WORKSPACE_NAME}

## Part 4 - Running Distributed PyTorch on GPUs

In this section we will be running a distributed, mult-node, GPU training job.  We'll be using the MNIST dataset and code and spread the training across two ranks.

For this training, we will be using _data parallel_ training.  This means that each rank has its own copy of the model and will be running on a different batch (in the process giving us twice the effective batch size).  Then, the ranks will use NCCL to perform an `all_reduce` collective operation which ensures all ranks have the final, averaged-out gradients and can apply them to their local copy of the model. The entire process is encapsulated by PyTorch's `DistributedDataParallel` class.

We will also be using the open-source PyTorch Lightning library, which is a useful wrapper on top of plain PyTorch, offering a number of benefits such as the `Trainer` object that encapsulates the training and validation loops.  

### Running the Code
From your shell, run the following command:


In [ ]:
az ml job create --file jobs/mnist_ddp.yaml --web \
    --resource-group ${RESOURCE_GROUP} --workspace-name ${WORKSPACE_NAME}

**Exercise**:  Try increasing the number of nodes and the processes per node and re-running the job

# Summary

In this lab, we delved into the foundational concepts of distributed deep learning, a critical area for scaling machine learning models across multiple devices and nodes. We began by exploring how Azure Machine Learning (AML) assigns ranks to individual processes, which is essential for managing and coordinating tasks in a distributed environment. This ranking system ensures that each process knows its role and can effectively contribute to the overall computation.

Next, we examined the use of collective communications libraries, which are pivotal for enabling efficient communication between processes. These libraries, when used in conjunction with `pytorch.distributed`, facilitate the exchange of information and synchronization among processes. This step is crucial for maintaining consistency and accuracy in the distributed training process.

Finally, we integrated these concepts using PyTorch Lightning, a high-level framework that simplifies the implementation of complex training workflows. By wrapping our distributed training setup with PyTorch Lightning, we were able to streamline the process, making it more manageable and easier to execute a full training run. This approach not only enhances productivity but also ensures that best practices are followed, leading to more robust and scalable deep learning models.
